In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from collections import Counter
import os

In [3]:
#Evaluation Metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score

In [4]:
os.chdir('/home/shrikrishna/Desktop/Coursera Machine Learning/Machine Learning: Classification/Week 5/')
os.getcwd()

'/home/shrikrishna/Desktop/Coursera Machine Learning/Machine Learning: Classification/Week 5'

In [5]:
#Read the loans data
loan_path = '/home/shrikrishna/Desktop/Coursera Machine Learning/\
Machine Learning: Classification/Week 3 Decision Tree'
loans = pd.read_csv(loan_path + '/lending-club-data/lending-club-data.csv')

/home/shrikrishna/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
train_index = pd.read_json('module-8-assignment-2-train-idx.json')
test_index = pd.read_json('module-8-assignment-2-test-idx.json')

In [7]:
train_data = loans.iloc[train_index[0], :]
test_data = loans.iloc[test_index[0], :]

In [8]:
pd.DataFrame(train_index).head()

,0
0,1
1,6
2,7
3,10
4,12


In [9]:
train_data.shape

(37224, 68)

In [10]:
test_data.shape

(9284, 68)

###  Early stopping methods for decision trees

1. Reached a maximum depth. (set by parameter max_depth).
2. Reached a minimum node size. (set by parameter min_node_size).
3. Don't split if the gain in error reduction is too small. (set by parameter min_error_reduction).

## Features

We will be considering only the following features

In [11]:
[col for col in train_data.columns if(col.startswith('bad'))]

['bad_loans']

In [12]:
#Create a new column named 'safe_loans' using the column 'bad_loans'
def create_safe(data):
    if('bad_loans' in data.columns):
        data['safe_loans'] = data['bad_loans'].apply(lambda x:1 if(x==0) else 0)
    

In [13]:
list(map(create_safe, [train_data, test_data]))

/home/shrikrishna/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


[None, None]

In [14]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'

In [15]:
train_data[features + [target]].dtypes

grade             object
term              object
home_ownership    object
emp_length        object
safe_loans         int64
dtype: object

In [16]:
test_data[features + [target]].dtypes

grade             object
term              object
home_ownership    object
emp_length        object
safe_loans         int64
dtype: object

## Missing Value Analysis

In [17]:
def find_missing_values(data):
    features = data.columns
    
    missing_value_count = data.isna().sum()
    
    missing_value_percentage = data.isna().sum()*100/data.shape[0]
    
    missing_data = pd.DataFrame({'Features': features,
                                 'Missing Value Count': missing_value_count,
                                 'Missing Value Percentage': missing_value_percentage},
                                 columns = ['Features', 'Missing Value Count',
                                            'Missing Value Percentage'])
    missing_data = missing_data.sort_values(by='Missing Value Percentage',
                                            ascending = False)
    
    #Set Features as the index
    missing_data = missing_data.set_index('Features')
    
    return missing_data

In [18]:
find_missing_values(train_data[features]).head()

,Missing Value Count,Missing Value Percentage
Features,,
emp_length,1443,3.876531
grade,0,0.000000
term,0,0.000000
home_ownership,0,0.000000


In [20]:
find_missing_values(test_data[features]).head()

,Missing Value Count,Missing Value Percentage
Features,,
emp_length,349,3.759156
grade,0,0.000000
term,0,0.000000
home_ownership,0,0.000000


In [21]:
set(train_data['emp_length'])

{'1 year',
 '10+ years',
 '2 years',
 '3 years',
 '4 years',
 '5 years',
 '6 years',
 '7 years',
 '8 years',
 '9 years',
 '< 1 year',
 nan}

In [22]:
#Replace the missing values in the 'emp_length' column with 0
train_data.loc[train_data['emp_length'].isna()==True, 'emp_length']  = '0'

/home/shrikrishna/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [23]:
test_data.loc[test_data['emp_length'].isna()==True, 'emp_length'] = '0'

/home/shrikrishna/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [24]:
find_missing_values(train_data[features]).head()

,Missing Value Count,Missing Value Percentage
Features,,
grade,0,0.0
term,0,0.0
home_ownership,0,0.0
emp_length,0,0.0


In [25]:
find_missing_values(test_data[features]).head()

,Missing Value Count,Missing Value Percentage
Features,,
grade,0,0.0
term,0,0.0
home_ownership,0,0.0
emp_length,0,0.0


In [26]:
set(train_data['emp_length'])

{'0',
 '1 year',
 '10+ years',
 '2 years',
 '3 years',
 '4 years',
 '5 years',
 '6 years',
 '7 years',
 '8 years',
 '9 years',
 '< 1 year'}

In [27]:
set(test_data['emp_length'])

{'0',
 '1 year',
 '10+ years',
 '2 years',
 '3 years',
 '4 years',
 '5 years',
 '6 years',
 '7 years',
 '8 years',
 '9 years',
 '< 1 year'}

## Transform categorical data into binary features

In [28]:
train_data[features].head()

,grade,term,home_ownership,emp_length
1,C,60 months,RENT,< 1 year
6,F,60 months,OWN,4 years
7,B,60 months,RENT,< 1 year
10,C,36 months,RENT,< 1 year
12,B,36 months,RENT,3 years


In [29]:
test_data[features].head()

,grade,term,home_ownership,emp_length
24,D,60 months,RENT,2 years
41,A,36 months,MORTGAGE,10+ years
60,F,60 months,RENT,4 years
93,D,60 months,RENT,10+ years
132,B,36 months,RENT,2 years


In [30]:
OHE_features = []
def to_categorical(data, features = features):
    global OHE_features
    #Convert the categorical features to One-Hot-Encoded features
    df = pd.get_dummies(data[features])
    OHE_features = df.columns.tolist()
    
    #Append the new encoded data frame to the orignal data frame
    data = pd.concat([data, df], axis=1)
    
    #Drop the orignal categorical variables
    if(set(features).intersection() == set(features)):
        data = data.drop(features, axis=1)
    
    return (data)

In [31]:
train_data, test_data = list(map(to_categorical, [train_data, test_data]))

In [32]:
OHE_features

['grade_A',
 'grade_B',
 'grade_C',
 'grade_D',
 'grade_E',
 'grade_F',
 'grade_G',
 'term_ 36 months',
 'term_ 60 months',
 'home_ownership_MORTGAGE',
 'home_ownership_OTHER',
 'home_ownership_OWN',
 'home_ownership_RENT',
 'emp_length_0',
 'emp_length_1 year',
 'emp_length_10+ years',
 'emp_length_2 years',
 'emp_length_3 years',
 'emp_length_4 years',
 'emp_length_5 years',
 'emp_length_6 years',
 'emp_length_7 years',
 'emp_length_8 years',
 'emp_length_9 years',
 'emp_length_< 1 year']

In [33]:
#Check whether the old features are present in the data
set(train_data.columns).intersection(features) == set() and \
set(test_data.columns).intersection(features) == set()

True

In [34]:
#Check whether the old features are present in the data
set(train_data.columns).intersection(features) != set() or \
set(test_data.columns).intersection(features) != set()

False

In [35]:
train_data[OHE_features].head()

,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,term_ 36 months,term_ 60 months,home_ownership_MORTGAGE,...,emp_length_10+ years,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year
1,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
6,0,0,0,0,0,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
7,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
10,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
12,0,1,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0


In [36]:
target in test_data.columns

True

In [37]:
binary_features = train_data.columns

In [38]:
target in train_data.columns

True

In [39]:
np.unique(train_data[target])

array([0, 1])

## Weighted decision trees

In [40]:
#Weighted error / Sum of weights  
def weighted_error(data_weights, actual, predicted):
    #Calculate the indicator for error
    error_indicator = [int(act==pred) for act, pred in zip(actual, predicted)]
    
    #Convert to numpy array
    data_weights, error_indicator = list(map(np.array,\
                                             [data_weights, error_indicator]))
    
    numerator = np.dot(data_weights, error_indicator)
    
    denominator = sum(data_weights)
    
    return (numerator/denominator)

In [41]:
act = [1, -1, 1, -1]
pred = [1, 1, 1, 1]
[int(act==pred) for act, pred in zip(act, pred)]

[1, 0, 1, 0]

## Calculate the Mistakes

The function intermediate_node_weighted_mistakes should first compute two weights:

    WM−1: weight of mistakes when all predictions are y^i=−1 i.e. WM(α,−1)
    WM+1: weight of mistakes when all predictions are y^i=+1 i.e. WM(α,+1)

where −1 and +1 are vectors where all values are -1 and +1 respectively.

After computing WM−1 and WM+1, the function intermediate_node_weighted_mistakes should return the lower of the two weights of mistakes, along with the class associated with that weight.

In [42]:
def get_index(data, label):
    #The data is a data frame with the target
    #print(data)
    return [ind for ind in data[target].index.tolist() if(data[target][ind]==label)]
            

In [43]:
l_ = train_data.loc[train_data['grade_A']==0, :]
temp_l_ = [ind for ind in l_[target].index.tolist() if(l_[target][ind]==0)]

In [44]:
r_ = train_data.loc[train_data['grade_A']==1, :]
temp_r_ = [ind for ind in r_[target].index.tolist() if(r_[target][ind]==0)]

In [61]:
set(temp_r_).intersection(set(temp_l_))

set()

In [46]:
temp_l_

[1,
 6,
 7,
 10,
 12,
 18,
 21,
 23,
 45,
 48,
 50,
 58,
 63,
 87,
 89,
 102,
 108,
 111,
 118,
 136,
 138,
 140,
 158,
 159,
 165,
 166,
 169,
 183,
 191,
 197,
 201,
 202,
 207,
 225,
 229,
 242,
 245,
 271,
 272,
 283,
 286,
 290,
 292,
 300,
 305,
 308,
 323,
 336,
 337,
 364,
 366,
 373,
 375,
 380,
 402,
 407,
 409,
 419,
 427,
 435,
 444,
 459,
 461,
 466,
 471,
 473,
 474,
 479,
 480,
 492,
 500,
 501,
 505,
 511,
 513,
 518,
 523,
 526,
 531,
 542,
 547,
 548,
 558,
 563,
 569,
 573,
 606,
 607,
 611,
 613,
 633,
 638,
 643,
 658,
 665,
 670,
 681,
 695,
 700,
 706,
 708,
 715,
 719,
 722,
 729,
 766,
 781,
 785,
 787,
 796,
 800,
 805,
 812,
 824,
 830,
 832,
 836,
 844,
 876,
 915,
 918,
 928,
 931,
 934,
 935,
 948,
 952,
 957,
 963,
 970,
 973,
 975,
 979,
 980,
 992,
 994,
 1006,
 1009,
 1011,
 1049,
 1053,
 1055,
 1089,
 1092,
 1093,
 1096,
 1100,
 1120,
 1122,
 1123,
 1129,
 1132,
 1147,
 1150,
 1179,
 1186,
 1190,
 1193,
 1214,
 1215,
 1222,
 1246,
 1249,
 1251,
 1289,

In [72]:
def intermediate_node_weighted_mistakes(labels_in_node, data_weights):
    #Sum the weights of all entries with label +1
    #labels_in_node is a data frame 
    #Obtain the indices where labels_in_node == +1
#     print("Label in Node:-\n{}" \
#          .format(labels_in_node.head()))
    
    positive_labels_indices = get_index(labels_in_node, 1)
    #print(positive_labels_indices)
    print("Are all the +ve labels in current split present in the data weights")
    print(list(set(positive_labels_indices).difference(\
                                            set(data_weights.index))))
    total_weight_positive = sum(data_weights[positive_labels_indices].values)
    
    # Weight of mistakes for predicting all -1's is equal to the sum above
    ### YOUR CODE HERE
    weighted_mistakes_all_negative = total_weight_positive
    
    # Sum the weights of all entries with label -1
    ### YOUR CODE HERE
    negative_labels_indices = list(set(labels_in_node.index).difference(\
                              set(positive_labels_indices)))
    print("Are all the -ve labels in current split present in the data weights")
    print(list(set(negative_labels_indices).difference(\
                                            set(data_weights.index))))
    total_weight_negative = sum(data_weights[negative_labels_indices].values)
    
    # Weight of mistakes for predicting all +1's is equal to the sum above
    ### YOUR CODE HERE
    weighted_mistakes_all_positive = total_weight_negative
    
    # Return the tuple (weight, class_label) representing the lower of the two weights
    #    class_label should be an integer of value +1 or -1.
    # If the two weights are identical, return (weighted_mistakes_all_positive,+1)
    ### YOUR CODE HERE
    if(total_weight_positive <= total_weight_negative):
        print("Weighted Mistake:- {}\tClass Label:- {}"\
              .format(weighted_mistakes_all_positive, 1))
        return (weighted_mistakes_all_positive, 1)
    else:
        print("Weighted Mistake:- {}\tClass Label:- {}"\
              .format(weighted_mistakes_all_negative, -1))
        return (total_weight_negative, -1)
    

## Quiz Question: If we set the weights α=1\alpha =1α=1 for all data points, how is the weight of mistakes WM(α,y^) related to the classification error?

In [48]:
print("Classification Error is equal to (Weight of mistakes WM(α,y^))/(Toal Number of Data Points)")

Classification Error is equal to (Weight of mistakes WM(α,y^))/(Toal Number of Data Points)


## 9. Follow these steps to implement best_splitting_feature:

    Step 1: Loop over each feature in the feature list
    
    Step 2: Within the loop, split the data into two groups: one group where all of the data has feature value 0 or False (we will call this the left split), and one group where all of the data has feature value 1 or True (we will call this the right split). Make sure the left split corresponds with 0 and the right split corresponds with 1 to ensure your implementation fits with our implementation of the tree building process.
    
    Step 3: Calculate the number of misclassified examples in both groups of data and use the above formula to compute the classification error.
    
    Step 4: If the computed error is smaller than the best error found so far, store this feature and its error.

## Function to pick best feature to split on

11. We continue modifying our decision tree code from the earlier assignment to incorporate weighting of individual data points. The next step is to pick the best feature to split on.

The best_splitting_feature function is similar to the one from the earlier assignment with two minor modifications:

    The function best_splitting_feature should now accept an extra parameter data_weights to take account of weights of data points.
    
    Instead of computing the number of mistakes in the left and right side of the split, we compute the weight of mistakes for both sides, add up the two weights, and divide it by the total weight of the data.

Your function should be analogous to the following Python function:



In [49]:
def split_on(feature, data_weights):
    intermediate_node_weighted_mistakes(labels_in_node, data_weights)

In [74]:
def best_splitting_feature(data, features, target, data_weights):
    
    # These variables will keep track of the best feature and the corresponding error
    best_feature = None
    best_error = float('+inf') 
    num_points = float(len(data))

    # Loop through each feature to consider splitting on that feature
    for feature in features:
        print("Feature:- " + feature)
        # The left split will have all data points where the feature value is 0
        # The right split will have all data points where the feature value is 1
        left_split = data.loc[data[feature] == 0, :]
        right_split = data.loc[data[feature] == 1, :]
        
        # Apply the same filtering to data_weights to create left_data_weights, right_data_weights
        ## YOUR CODE HERE 
        ##Sum of weights of the data points selected to be on the left
        left_data_weights = data_weights[left_split\
                                        .index.tolist()]
        
        ##Sum of weights of the data points selected to be on the right
        right_data_weights = data_weights[right_split\
                                        .index.tolist()]
                    
        # DIFFERENT HERE
        # Calculate the weight of mistakes for left and right sides
        ## YOUR CODE HERE
        left_weighted_mistakes, left_class = intermediate_node_weighted_mistakes\
                                             (left_split, left_data_weights)
        right_weighted_mistakes, right_class = intermediate_node_weighted_mistakes\
                                             (right_split, right_data_weights)
        
        # DIFFERENT HERE
        # Compute weighted error by computing
        #  ( [weight of mistakes (left)] + [weight of mistakes (right)] ) / [total weight of all data points]
        ## YOUR CODE HERE
        error = (left_weighted_mistakes + right_weighted_mistakes)/len(data)
        
        # If this is the best error we have found so far, store the feature and the error
        if error < best_error:
            best_feature = feature
            best_error = error
    
    # Return the best feature we found
    return best_feature

{ 
   'is_leaf'            : True/False.
   'prediction'         : Prediction at the leaf node.
   'left'               : (dictionary corresponding to the left tree).
   'right'              : (dictionary corresponding to the right tree).
   'splitting_feature'  : The feature that this node splits on
}


In [66]:
def create_leaf(data, data_weights):
    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'is_leaf': True}
    
    # Computed weight of mistakes.
    # Store the predicted class (1 or -1) in leaf['prediction']
    weighted_error, best_class = intermediate_node_weighted_mistakes(data, data_weights)
    leaf['prediction'] = best_class ## YOUR CODE HERE
    
    return leaf

We have provided a function that learns the decision tree recursively 
and implements 3 stopping conditions:

    Stopping condition 1: All data points in a node are from the same class.
    
    Stopping condition 2: No more features to split on.
    
    Additional stopping condition: In addition to the above two stopping conditions covered in lecture, in this assignment we will also consider a stopping condition based on the max_depth of the tree. By not letting the tree grow too deep,
    
    we will save computational effort in the learning process.

In [52]:
target_values = train_data[target]
target_values = target_values.reset_index()
if('index' in target_values.columns):
        target_values = target_values.drop(['index'], axis=1)
        
target_values.loc[target_values[target]==1, :]

,safe_loans
18476,1
18477,1
18478,1
18479,1
18480,1
18481,1
18482,1
18483,1
18484,1
18485,1


In [67]:
def weighted_decision_tree_create(data, features, target, data_weights, current_depth = 1, max_depth = 10):
    remaining_features = features[:] # Make a copy of the features.
    target_values = data[target]
    print ("--------------------------------------------------------------------")
    print ("Subtree, depth = %s (%s data points)." % (current_depth, len(target_values)))
    
#     #Reset the index of the target_values so that it matches with the index of data_weights
#     target_values = target_values.reset_index()
#     #Drop the redundant index column
#     if('index' in target_values.columns):
#         target_values = target_values.drop(['index'], axis=1)
    
    # Stopping condition 1. Error is 0.
    # Weighted Error is close to 0.
    if intermediate_node_weighted_mistakes(data, data_weights)[0] <= 1e-15:
        print ("Stopping condition 1 reached."              )
        return create_leaf(data, data_weights)
    
    #Stopping condition:- If all the data points are from the same class
    if (len(np.unique(target_values)) == 1):
        print ("All the data points of the split belong to the same class.")
        return create_leaf(data, data_weights)
    
    # Stopping condition 2. No more features.
    if remaining_features == []:
        print ("Stopping condition 2 reached."               ) 
        return create_leaf(data, data_weights)    
    
    # Additional stopping condition (limit tree depth)
    if current_depth > max_depth:
        print ("Reached maximum depth. Stopping for now.")
        return create_leaf(data, data_weights)
    
    # If all the datapoints are the same, splitting_feature will be None. Create a leaf
    splitting_feature = best_splitting_feature(data, features, target, data_weights)
    if(not splitting_feature):
        return create_leaf(data, data_weights)
    
    remaining_features.remove(splitting_feature)
        
    left_split = data.loc[data[splitting_feature] == 0, :]
    right_split = data.loc[data[splitting_feature] == 1, :]
    
    left_data_weights = data_weights[left_split.\
                                    index.tolist()]
    right_data_weights = data_weights[right_split.\
                                    index.tolist()]
    
    print ("Split on feature %s. (%s, %s)" % (\
              splitting_feature, len(left_split), len(right_split)))
    
    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        print ("Creating leaf node.")
        return create_leaf(left_split, data_weights)
    
    if len(right_split) == len(data):
        print ("Creating leaf node.")
        return create_leaf(right_split, data_weights)
    
    # Repeat (recurse) on left and right subtrees
    left_tree = weighted_decision_tree_create(
        left_split, remaining_features, target, left_data_weights, current_depth + 1, max_depth)
    right_tree = weighted_decision_tree_create(
        right_split, remaining_features, target, right_data_weights, current_depth + 1, max_depth)
    
    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

## Count the number of nodes in the tree

In [54]:
def get_num_nodes(tree):
    if(tree['is_leaf']):
        return 1
    return (1 + get_num_nodes(tree['left']) + get_num_nodes(tree['right']))

### Making predictions with a weighted decision tree

15. To make a single prediction, we must start at the root and traverse down the decision tree in recursive fashion. Write a function classify that makes a single prediction. It should be analogous to the following:

In [55]:
def classify(tree, x, annotate = False):
    #if the node is a leaf node.
    if tree['is_leaf']:
        if annotate:
            print ("At leaf, predicting %s" % tree['prediction'])
        return tree['prediction']
    
    else:
        #split on feature.
        #This is chosen as the best feature to split on
        split_feature_value = x[tree['splitting_feature']]
        
        if annotate:
             print ("Split on %s = %s" % (tree['splitting_feature'],\
                                          split_feature_value))
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        
        else:
            ### YOUR CODE HERE
            return classify(tree['right'], x, annotate)
               

### Evaluating the tree

16. Create a function called evaluate_classification_error. It takes in as input:

    tree (as described above)
    data (an data frame)

The function does not change because of adding data point weights. It is analogous to this Python function:


In [56]:
def evaluate_classification_error(tree, data):
    # Apply the classify(tree, x) to each row in your data
    prediction = data.apply(lambda x: classify(tree, x))
    
    # Once you've made the predictions, calculate the classification error
    clf_error = sum([act!= pred for act, pred in\
                     zip(prediction, data[target])]) / float(len(data))
    
    return clf_error

### 17. To build intuition on how weighted data points affect the tree being built, consider the following:

Suppose we only care about making good predictions for the first 10 and last 10 items in train_data, we assign weights:

    1 to the last 10 items
    1 to the first 10 items
    and 0 to the rest.

Let us fit a weighted decision tree with max_depth = 2. Then compute the classification error on the subset_20, i.e. the subset of data points whose weight is 1 (namely the first and last 10 data points).


In [57]:
train_data.loc[train_data[target]==1, target]

22        1
26        1
28        1
30        1
31        1
34        1
44        1
49        1
53        1
57        1
59        1
61        1
86        1
88        1
90        1
96        1
104       1
112       1
113       1
119       1
120       1
131       1
137       1
146       1
154       1
157       1
163       1
176       1
185       1
186       1
         ..
122433    1
122436    1
122454    1
122458    1
122465    1
122473    1
122484    1
122489    1
122494    1
122498    1
122500    1
122501    1
122505    1
122506    1
122513    1
122520    1
122525    1
122531    1
122538    1
122544    1
122552    1
122553    1
122558    1
122561    1
122569    1
122572    1
122575    1
122588    1
122599    1
122603    1
Name: safe_loans, Length: 18748, dtype: int64

In [58]:
train_data[target].index.tolist()

[1,
 6,
 7,
 10,
 12,
 18,
 21,
 23,
 45,
 48,
 50,
 58,
 63,
 87,
 89,
 102,
 108,
 111,
 118,
 124,
 136,
 138,
 140,
 151,
 158,
 159,
 165,
 166,
 169,
 183,
 191,
 197,
 201,
 202,
 207,
 225,
 229,
 238,
 242,
 245,
 250,
 271,
 272,
 283,
 286,
 290,
 292,
 300,
 301,
 305,
 308,
 316,
 323,
 336,
 337,
 344,
 364,
 366,
 373,
 375,
 380,
 402,
 407,
 409,
 419,
 423,
 427,
 430,
 435,
 444,
 459,
 461,
 466,
 468,
 471,
 473,
 474,
 479,
 480,
 492,
 493,
 496,
 500,
 501,
 505,
 510,
 511,
 513,
 518,
 523,
 526,
 531,
 542,
 547,
 548,
 558,
 563,
 569,
 573,
 606,
 607,
 611,
 613,
 633,
 638,
 643,
 658,
 665,
 670,
 678,
 681,
 692,
 695,
 700,
 706,
 708,
 715,
 719,
 722,
 729,
 757,
 766,
 781,
 785,
 787,
 796,
 800,
 805,
 812,
 824,
 830,
 832,
 836,
 844,
 868,
 876,
 891,
 915,
 918,
 928,
 931,
 934,
 935,
 948,
 950,
 952,
 957,
 963,
 967,
 970,
 973,
 975,
 979,
 980,
 992,
 994,
 1006,
 1009,
 1011,
 1017,
 1049,
 1051,
 1053,
 1055,
 1067,
 1083,
 1089,
 1092

In [73]:
# Assign weights
example_data_weights = pd.Series([1.] * 10 + [0.]*(len(train_data) - 20) + [1.] * 10)
# Train a weighted decision tree model.
small_data_decision_tree_subset_20 = weighted_decision_tree_create(train_data, OHE_features, target,
                         example_data_weights, max_depth=2)

--------------------------------------------------------------------
Subtree, depth = 1 (37224 data points).
Are all the +ve labels in current split present in the data weights
[65546, 65554, 65555, 98323, 65560, 65564, 98334, 98341, 65574, 65583, 65587, 98355, 65602, 98373, 98375, 98376, 98377, 98385, 65619, 65622, 65623, 65624, 98395, 98398, 65635, 65643, 65646, 98415, 98417, 65653, 98422, 65667, 98436, 98438, 65673, 98441, 98447, 65681, 65686, 98454, 98459, 65693, 65698, 98473, 65714, 65730, 98500, 65734, 98503, 98514, 98516, 65749, 98528, 98534, 65768, 98537, 65773, 98543, 65776, 65783, 98555, 65787, 98556, 98557, 65793, 98564, 98565, 98566, 65809, 65811, 65812, 98583, 65817, 98585, 98592, 98598, 98605, 98608, 65842, 98613, 98614, 98617, 65850, 98624, 98625, 98627, 65866, 65872, 98650, 65883, 98657, 98673, 98674, 98675, 65908, 98678, 98690, 65931, 98701, 65938, 65948, 98717, 65952, 65953, 98721, 65955, 98729, 65963, 65966, 98740, 98742, 98744, 98745, 98753, 98754, 65987, 98755, 987

/home/shrikrishna/anaconda3/lib/python3.6/site-packages/pandas/core/series.py:841: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


Are all the +ve labels in current split present in the data weights
[]
Are all the -ve labels in current split present in the data weights
[]
Weighted Mistake:- nan	Class Label:- -1
Are all the +ve labels in current split present in the data weights
[]
Are all the -ve labels in current split present in the data weights
[]
Weighted Mistake:- nan	Class Label:- -1
Feature:- grade_B
Are all the +ve labels in current split present in the data weights
[]
Are all the -ve labels in current split present in the data weights
[]
Weighted Mistake:- nan	Class Label:- -1
Are all the +ve labels in current split present in the data weights
[]
Are all the -ve labels in current split present in the data weights
[]
Weighted Mistake:- nan	Class Label:- -1
Feature:- grade_C
Are all the +ve labels in current split present in the data weights
[]
Are all the -ve labels in current split present in the data weights
[]
Weighted Mistake:- nan	Class Label:- -1
Are all the +ve labels in current split present in the

Are all the +ve labels in current split present in the data weights
[]
Are all the -ve labels in current split present in the data weights
[]
Weighted Mistake:- nan	Class Label:- -1
Are all the +ve labels in current split present in the data weights
[]
Are all the -ve labels in current split present in the data weights
[]
Weighted Mistake:- nan	Class Label:- -1
Feature:- emp_length_9 years
Are all the +ve labels in current split present in the data weights
[]
Are all the -ve labels in current split present in the data weights
[]
Weighted Mistake:- nan	Class Label:- -1
Are all the +ve labels in current split present in the data weights
[]
Are all the -ve labels in current split present in the data weights
[]
Weighted Mistake:- nan	Class Label:- -1
Feature:- emp_length_< 1 year
Are all the +ve labels in current split present in the data weights
[]
Are all the -ve labels in current split present in the data weights
[]
Weighted Mistake:- nan	Class Label:- -1
Are all the +ve labels in curre

In [60]:
example_data_weights = pd.Series([1.0 for i in range(len(train_data))])
small_data_decision_tree = weighted_decision_tree_create(train_data, OHE_features, target,
                                        example_data_weights, max_depth=2)
if count_nodes(small_data_decision_tree) == 7:
    print ('Test passed!')
else:
    print ('Test failed... try again!')
    print ('Number of nodes found:', count_nodes(small_data_decision_tree))
    print ('Number of nodes that should be there: 7' )

--------------------------------------------------------------------
Subtree, depth = 1 (37224 data points).


/home/shrikrishna/anaconda3/lib/python3.6/site-packages/pandas/core/series.py:841: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


Weighted Mistake:- nan	Class Label:- -1
Feature:- grade_A
Weighted Mistake:- nan	Class Label:- -1
Weighted Mistake:- nan	Class Label:- -1
Feature:- grade_B
Weighted Mistake:- nan	Class Label:- -1
Weighted Mistake:- nan	Class Label:- -1
Feature:- grade_C
Weighted Mistake:- nan	Class Label:- -1
Weighted Mistake:- nan	Class Label:- -1
Feature:- grade_D
Weighted Mistake:- nan	Class Label:- -1
Weighted Mistake:- nan	Class Label:- -1
Feature:- grade_E
Weighted Mistake:- nan	Class Label:- -1
Weighted Mistake:- nan	Class Label:- -1
Feature:- grade_F
Weighted Mistake:- nan	Class Label:- -1
Weighted Mistake:- nan	Class Label:- -1
Feature:- grade_G
Weighted Mistake:- nan	Class Label:- -1
Weighted Mistake:- nan	Class Label:- -1
Feature:- term_ 36 months
Weighted Mistake:- nan	Class Label:- -1
Weighted Mistake:- nan	Class Label:- -1
Feature:- term_ 60 months
Weighted Mistake:- nan	Class Label:- -1
Weighted Mistake:- nan	Class Label:- -1
Feature:- home_ownership_MORTGAGE
Weighted Mistake:- nan	Class

NameError: name 'count_nodes' is not defined

In [ ]:
train_data.index

In [83]:
def count_nodes(tree):
    if tree['is_leaf']:
        return 1
    return 1 + count_nodes(tree['left']) + count_nodes(tree['right'])

### 18. Now, we will compute the classification error on the subset_20, i.e. the subset of data points whose weight is 1 (namely the first and last 10 data points). 

In [77]:
evaluate_classification_error(small_data_decision_tree_subset_20, train_data)

0.0024177949709864605

### Subset 20 data

In [87]:
#Evaluate on subset_20 data
#Select only those weights which are 1
subset_20_data_weights = pd.concat([example_data_weights[:10],\
                                  example_data_weights[-10:]])

#Select the data corresponding to these weights
subset_20_data = pd.concat([train_data.iloc[:10, :], train_data.iloc[-10:, :]])


In [88]:
subset_20_decision_tree = weighted_decision_tree_create(\
                                    subset_20_data, features, target,
                                    subset_20_data_weights, max_depth=2)



--------------------------------------------------------------------
Subtree, depth = 1 (20 data points).


KeyError: 'None of [[122552, 122553, 122558, 122561, 122569, 122572, 122575, 122588, 122599, 122603]] are in the [index]'

In [89]:
evaluate_classification_error(small_data_decision_tree_subset_20, train_data)

0.0024177949709864605

### Early stopping condition 2: Minimum node size

In [90]:
def reached_minimum_node_size(data, min_node_size):
    # Return True if the number of data points is less than or equal to the minimum node size.
    ## YOUR CODE HERE
    if(data.shape[0] <= min_node_size):
        return True

### Quiz question: Given an intermediate node with 6 safe loans and 3 risky loans, if the min_node_size parameter is 10, what should the tree learning algorithm do next?

In [91]:
create_leaf(pd.Series([1, 1, 1, 1, 1, 1, -1, -1, -1]))

TypeError: create_leaf() missing 1 required positional argument: 'data_weights'

## Early stopping condition 3: Minimum gain in error reduction

The function error_reduction takes 2 arguments:

    The error before a split, error_before_split.
    The error after a split, error_after_split.

In [92]:
def error_reduction(error_before_split, error_after_split):
    # Return the error before the split minus the error after the split.
    ## YOUR CODE HERE
    return (error_before_split - error_after_split)

In [93]:
def decision_tree_create(data, features, target, current_depth = 0,\
                         max_depth = 10, min_node_size=1, 
                         min_error_reduction=0.0):
    remaining_features = features[:] # Make a copy of the features.
    #remaining_features = remaining_features.tolist()
    
    target_values = data[target]
    print ("--------------------------------------------------------------------")
    print ("Subtree, depth = %s (%s data points)." % (current_depth,\
                                                      len(target_values)\
                                                     ))

    # Stopping condition 1
    # (Check if there are mistakes at current node.)
    # Recall you wrote a function intermediate_node_num_mistakes to compute this.
    if (intermediate_node_num_mistakes(target_values)) == 0:  ## YOUR CODE HERE
        print ("Stopping condition 1 reached.")
        
        # If no mistakes at current node, make current node a leaf node
        return create_leaf(target_values)
    
    # Stopping condition 2 (check if there are remaining features to consider splitting on)
    if (len(remaining_features) == 0):   ## YOUR CODE HERE
        print ("Stopping condition 2 reached.")    
        # If there are no remaining features to consider, make current node a leaf node
        return create_leaf(target_values)    
    
    # Additional stopping condition (3) (limit tree depth)
    if (current_depth >= max_depth):  ## YOUR CODE HERE
        print ("Reached maximum depth. Stopping for now.")
        # If the max tree depth has been reached, make current node a leaf node
        return create_leaf(target_values)
    
    # Early stopping condition 2: Reached the minimum node size.
    # If the number of data points is less than or equal to the minimum size, return a leaf.
    if (reached_minimum_node_size(data, min_node_size)): ## YOUR CODE HERE 
        print ("Early stopping condition 2 reached.\
                Reached minimum node size.")
        return create_leaf(target_values)  ## YOUR CODE HERE
    
    # Find the best splitting feature
    splitting_feature = best_splitting_feature(data, features, target)
    
    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    
    # Early stopping condition 3: Minimum error reduction
    # Calculate the error before splitting (number of misclassified examples 
    # divided by the total number of examples)
    error_before_split = intermediate_node_num_mistakes(target_values) \
                                    / float(len(data))
    
    # Calculate the error after splitting (number of misclassified examples 
    # in both groups divided by the total number of examples)
    left_mistakes = intermediate_node_num_mistakes(left_split)  ## YOUR CODE HERE
    right_mistakes =  intermediate_node_num_mistakes(right_split) ## YOUR CODE HERE
    error_after_split = (left_mistakes + right_mistakes) \
                                    / float(len(data))
    
    # If the error reduction is LESS THAN OR EQUAL TO min_error_reduction
    # , return a leaf.
    reduction_in_error = error_reduction(error_before_split,\
                                      error_after_split)
    if (reduction_in_error <= min_error_reduction): ## YOUR CODE HERE
        print ("Early stopping condition 3 reached.\
                Minimum error reduction.")
        return create_leaf(target_values) ## YOUR CODE HERE 
    
    # Find the best splitting feature (recall the function best_splitting_feature implemented above)
    ## YOUR CODE HERE
    splitting_feature = best_splitting_feature(data, remaining_features,\
                                               target)
    #splitting_feature contains the feature value in the form of string
    
    # Split on the best feature that we found. 
    left_split = data.loc[data[splitting_feature] == 0]
    
    ## YOUR CODE HERE
    right_split = data.loc[data[splitting_feature] == 1]      
    
    remaining_features.remove(splitting_feature)
    print ("Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split),\
                                             len(right_split)))
    
    # Create a leaf node if the split is "perfect"
    if (len(left_split) == len(data)):
        print ("Creating leaf node.")
        return create_leaf(left_split[target])
    if (len(right_split) == len(data)):
        print ("Creating leaf node.")
        ## YOUR CODE HERE
        return create_leaf(right_split[target])
    
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features,\
                                     target, current_depth + 1, max_depth)        
    ## YOUR CODE HERE
    right_tree = decision_tree_create(right_split, remaining_features,\
                                      target, current_depth + 1, max_depth)

    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

In [94]:
OHE_features

['grade_A',
 'grade_B',
 'grade_C',
 'grade_D',
 'grade_E',
 'grade_F',
 'grade_G',
 'term_ 36 months',
 'term_ 60 months',
 'home_ownership_MORTGAGE',
 'home_ownership_OTHER',
 'home_ownership_OWN',
 'home_ownership_RENT',
 'emp_length_0',
 'emp_length_1 year',
 'emp_length_10+ years',
 'emp_length_2 years',
 'emp_length_3 years',
 'emp_length_4 years',
 'emp_length_5 years',
 'emp_length_6 years',
 'emp_length_7 years',
 'emp_length_8 years',
 'emp_length_9 years',
 'emp_length_< 1 year']

In [95]:
small_data_decision_tree = decision_tree_create(train_data, OHE_features, 'safe_loans', max_depth = 3)
if count_nodes(small_data_decision_tree) == 13:
    print ('Test passed!')
else:
    print ('Test failed... try again!')
    print ('Number of nodes found                :', count_nodes(small_data_decision_tree))
    print ('Number of nodes that should be there : 13')

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).


NameError: name 'intermediate_node_num_mistakes' is not defined

In [96]:
for fea in features:
    print(fea in train_data.columns)

False
False
False
False


## Build a tree!

13. Now that your code is working, train a tree model on the train_data with

    max_depth = 6
    min_node_size = 100,
    min_error_reduction = 0.0



In [97]:
target

'safe_loans'

In [98]:
target in train_data.columns

True

In [99]:
temp = binary_features[:].copy()

In [100]:
len(OHE_features)

25

In [ ]:
binary_features

In [ ]:
#train_data[target] = loans.loc[train_index[0], target]
#test_data[target] = loans.iloc[test_index[0], target]
#With Early Stopping Conditions
my_decision_tree_new = decision_tree_create(train_data, OHE_features,\
                                        target, max_depth = 6,\
                                        min_node_size = 100,\
                                        min_error_reduction=0.0)

### 14. Let's now train a tree model ignoring early stopping conditions 2 and 3 so that we get the same tree as in the previous assignment. To ignore these conditions, we set min_node_size=0 and min_error_reduction=-1 (a negative value). Call this model my_decision_tree_old.


In [ ]:
my_decision_tree_old = decision_tree_create(train_data, OHE_features,\
                                            target,\
                                            max_depth = 6,\
                                            min_node_size = 0,\
                                            min_error_reduction=-1)

###  Now, let's consider the first example of the test set and see what my_decision_tree model predicts for this data point.


In [ ]:
test_data[OHE_features].head()

In [ ]:
train_data, test_data = list(map(lambda x:x.reset_index(),\
                                 [train_data, test_data]))


In [ ]:
#print (test_data.loc[0, OHE_features])
print ('Predicted class: %s ' % classify(my_decision_tree_new,\
                                         test_data.loc[0, OHE_features]))


In [ ]:
test_data[OHE_features].head()

In [ ]:
my_decision_tree_new

In [ ]:
#Prediction Path of New Decision tree for the test data.
classify(my_decision_tree_new, test_data.loc[0, OHE_features], annotate = True)

In [ ]:
my_decision_tree_old

In [ ]:
#Prediction Path of Old Decision Tree for the test data
classify(my_decision_tree_old, test_data.loc[0, OHE_features],\
         annotate = True)

### Quiz question: For my_decision_tree_new trained with max_depth = 6, min_node_size = 100, min_error_reduction=0.0, is the prediction path for validation_set[0] shorter, longer, or the same as for my_decision_tree_old that ignored the early stopping conditions 2 and 3?

In [ ]:
print("Same Path")

### Quiz question: For my_decision_tree_new trained with max_depth = 6, min_node_size = 100, min_error_reduction=0.0, is the prediction path for any point always shorter, always longer, always the same, shorter or the same, or longer or the same as for my_decision_tree_old that ignored the early stopping conditions 2 and 3?

In [ ]:
print("Shorter or same")

In [ ]:
def classify_util(tree, data):
    prediction_test = []
    for i in range(len(data)):
        result = classify(tree, data.loc[i, OHE_features],\
                      annotate=True)
        prediction_test.append(result)
    return (prediction_test)

In [ ]:
#Prediction using old tree on the whole test data
prediction_test_old = classify_util(my_decision_tree_old, test_data)

In [ ]:
#Prediction using new tree on the whole test data
prediction_test_new = classify_util(my_decision_tree_new, test_data)

### Quiz question: For a tree trained on any dataset using max_depth = 6, min_node_size = 100, min_error_reduction=0.0, what is the maximum number of splits encountered while making a single prediction?

In [ ]:
r = 2
a = 2
n = 6
print((a * (r**n - 1))/(r - 1))

In [ ]:
sum([2**i for i in range(1, 7)])

In [ ]:
sum([2**i for i in range(6)])

In [ ]:
126 + 1 - (2**6)

In [ ]:
63 + 64

## Evaluating your decision tree

16. Now, we will write a function to evaluate a decision tree by computing the classification error of the tree on the given dataset. Write a function called evaluate_classification_error that takes in as input:

    tree (as described above)
    data (a data frame of data points)

In [ ]:
def evaluate_classification_error(tree, data):
    # Apply the classify(tree, x) to each row in your data
    #prediction = classify(my_decision_tree, test_data[OHE_features], annotate=True)
    prediction = classify_util(tree, data)
    
    # Once you've made the predictions, calculate the classification error and return it
    no_mistakes = len([act for act, pred in zip(data[target], prediction) if(act!=pred)])
    num_data_points = float(len(data))
    classification_error = no_mistakes/num_data_points
    
    return (classification_error)

### 20. Now, let's use this function to evaluate the classification error of my_decision_tree_new on the validation_set. Your code should be analogous to

In [ ]:
clf_error_new = evaluate_classification_error(my_decision_tree_new, test_data)

In [ ]:
clf_error_new

### 21. Now, evaluate the validation error using my_decision_tree_old.

In [ ]:
clf_error_old = evaluate_classification_error(my_decision_tree_old, test_data)

In [ ]:
clf_error_old

In [ ]:
clf_error_new == clf_error_old

### Quiz question: Is the validation error of the new decision tree (using early stopping conditions 2 and 3) lower than, higher than, or the same as that of the old decision tree from the previous assignment?


In [ ]:
print("Same")

### Exploring the effect of max_depth

We will compare three models trained with different values of the stopping criterion. We intentionally picked models at the extreme ends (too small, just right, and too large).

22. Train three models with these parameters:

    model_1: max_depth = 2 (too small)
    
    model_2: max_depth = 6 (just right)
    
    model_3: max_depth = 14 (may be too large)

In [ ]:
#model_1: max_depth = 2 (too small)
model_1 = decision_tree_create(train_data, OHE_features,\
                                            target,\
                                            max_depth = 2,\
                                            min_node_size = 0,\
                                            min_error_reduction=-1)

In [ ]:
#model_2: max_depth = 6 (just right)
model_2 = decision_tree_create(train_data, OHE_features,\
                                            target,\
                                            max_depth = 6,\
                                            min_node_size = 0,\
                                            min_error_reduction=-1)

In [ ]:
#model_3: max_depth = 14 (may be too large)
model_3 = decision_tree_create(train_data, OHE_features,\
                                            target,\
                                            max_depth = 14,\
                                            min_node_size = 0,\
                                            min_error_reduction=-1)

### Evaluation on Validation Data

In [ ]:
print ("Training data, classification error (model 1):", evaluate_classification_error(model_1, train_data))


In [ ]:
print("Training data, classification error (model 1): 0.4963464431549538")

In [ ]:
print ("Training data, classification error (model 2):", evaluate_classification_error(model_2, train_data))

In [ ]:
print("Training data, classification error (model 2): 0.4963464431549538")

In [ ]:
#print ("Training data, classification error (model 3):", evaluate_classification_error(model_3, train_data))

In [ ]:
print("Training data, classification error (model 3): 0.4963464431549538")

### Evaluate the Classification Error on Validation Data

In [ ]:
#Model 1
print ("Training data, classification error (model 1):", evaluate_classification_error(model_1, test_data))

In [ ]:
print("Training data, classification error (model 1): 0.503446790176648")

In [ ]:
#Model 2
print ("Training data, classification error (model 2):", evaluate_classification_error(model_2, test_data))

In [ ]:
print("Validation data, classification error (model 2): 0.503446790176648")

In [ ]:
#Model 3
print ("Validation data, classification error (model 3):", evaluate_classification_error(model_3, test_data))

In [ ]:
print("Validation data, classification error (model 3): 0.503446790176648")

### Measure the Complexity of the Tree
Complexity(T) = Number of leaves in the tree

In [ ]:
def count_leaves(tree):
    if tree['is_leaf']:
        return 1
    return count_leaves(tree['left']) + count_leaves(tree['right'])

In [ ]:
leaves_1, leaves_2, leaves_3 = list(map(count_leaves, [model_1, model_2, model_3]))

In [ ]:
print("Complexity of Models\nModel_1:- %d\nModel_2:- %d\n\
Model_3:- %d\n" %(leaves_1, leaves_2, leaves_3))


### Exploring the effect of min_error

We will compare three models trained with different values of the stopping criterion. We intentionally picked models at the extreme ends (negative, just right, and too positive).

27. Train three models with these parameters:

    model_4: min_error_reduction = -1 (ignoring this early stopping condition)
    
    model_5: min_error_reduction = 0 (just right)
    
    model_6: min_error_reduction = 5 (too positive)

For each of these three, we set max_depth = 6, and min_node_size = 0. Make sure to call the models model_4, model_5, and model_6.

In [ ]:
#model_4: min_error_reduction = -1 (ignoring this early stopping condition)
model_4 = decision_tree_create(train_data, OHE_features,\
                                            target,\
                                            max_depth = 6,\
                                            min_node_size = 0,\
                                            min_error_reduction=-1)

In [ ]:
#model_5: min_error_reduction = 0 (just right)
model_5 = decision_tree_create(train_data, OHE_features,\
                                            target,\
                                            max_depth = 6,\
                                            min_node_size = 0,\
                                            min_error_reduction=0)

In [ ]:
# model_6: min_error_reduction = 5 (too positive)
model_6 = decision_tree_create(train_data, OHE_features,\
                                            target,\
                                            max_depth = 6,\
                                            min_node_size = 0,\
                                            min_error_reduction=5)

### Evaluation of Model 4, 5, and 6

In [ ]:
#Model 4
print ("Validation data, classification error (model 4):", evaluate_classification_error(model_4, test_data))

In [ ]:
print("Validation data, classification error (model 4): 0.503446790176648")

In [ ]:
#Model 5
print ("Validation data, classification error (model 5):", evaluate_classification_error(model_5, test_data))

In [ ]:
print("Validation data, classification error (model 5): 0.503446790176648")

In [ ]:
#Model 6
print ("Validation data, classification error (model 6):", evaluate_classification_error(model_6, test_data))

In [ ]:
print("Validation data, classification error (model 6): 0.503446790176648")

### Quiz Question: Using the complexity definition above, which model (model_4, model_5, or model_6) has the largest complexity? Did this match your expectation?

In [ ]:
leaves_4, leaves_5, leaves_6 = list(map(count_leaves, [model_4, model_5, model_6]))

In [ ]:
print("Complexity of Models\nModel_4:- %d\nModel_5:- %d\n\
Model_6:- %d\n" %(leaves_4, leaves_5, leaves_6))

### Quiz Question: model_4 and model_5 have similar classification error on the validation set but model_5 has lower complexity? Should you pick model_5 over model_4?

In [ ]:
print("Choosing the simplest model, model_4")

## Exploring the effect of min_node_size

We will compare three models trained with different values of the stopping criterion. Again, intentionally picked models at the extreme ends (too small, just right, and just right).

30. Train three models with these parameters:

    model_7: min_node_size = 0 (too small)
    
    model_8: min_node_size = 2000 (just right)
    
    model_9: min_node_size = 50000 (too large)

For each of these three, we set max_depth = 6, and min_error_reduction = -1. Make sure to call these models model_7, model_8, and model_9.

In [ ]:
# model_7: min_node_size = 0 (too small)
model_7 = decision_tree_create(train_data, OHE_features,\
                                            target,\
                                            max_depth = 6,\
                                            min_node_size = 0,\
                                            min_error_reduction=-1)

In [ ]:
#model_8: min_node_size = 2000 (just right)
model_8 = decision_tree_create(train_data, OHE_features,\
                                            target,\
                                            max_depth = 6,\
                                            min_node_size = 2000,\
                                            min_error_reduction=-1)

In [ ]:
#model_9: min_node_size = 50000 (too large)
model_9 = decision_tree_create(train_data, OHE_features,\
                                            target,\
                                            max_depth = 6,\
                                            min_node_size = 5000,\
                                            min_error_reduction=-1)

### 31. Calculate the accuracy of each model (model_7, model_8, or model_9) on the validation set.



In [ ]:
#Model 7
print ("Validation data, classification error (model 7):", evaluate_classification_error(model_7, test_data))

In [ ]:
#Model 8 Validat
print ("Validation data, classification error (model 8):", evaluate_classification_error(model_8, test_data))

In [ ]:
#Model 9
print ("Validation data, classification error (model 9):", evaluate_classification_error(model_9, test_data))

### 32. Using the count_leaves function, compute the number of leaves in each of each models (model_7, model_8, and model_9).

In [ ]:
leaves_7, leaves_8, leaves_9 = list(map(count_leaves, [model_7, model_8, model_9]))

In [ ]:
print("Complexity of Models\nModel_7:- %d\nModel_8:- %d\n\
Model_9:- %d\n" %(leaves_7, leaves_8, leaves_9))

### Quiz Question: Using the results obtained in this section, which model (model_7, model_8, or model_9) would you choose to use?